this script prepares the instrumental variable:

- total funding grants of each country in each year
- total funding grants from previous year

In [16]:
import pandas as pd
import numpy as np

In [3]:
pub_path = "../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"

In [4]:
df = pd.read_pickle(pub_path)

In [5]:
df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[France, Chile]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


derive the total number of funding grants from each country across years

In [20]:
fund_total = df[df.funder!='Not-Funded'][['funder','year']].explode('funder'
                                                ).groupby(['funder','year']).size().reset_index(name='fundcnt')

In [21]:
topcntrys=['United States','China','EU','United Kingdom','France','Germany']
fund_total['region'] = fund_total['funder'].apply(lambda x: x if x in topcntrys else "others")
fund_total = fund_total.groupby(['region','year'])['fundcnt'].sum().reset_index()

In [22]:
fund_total = fund_total.pivot(index='year',columns='region',values='fundcnt')
fund_total.columns = fund_total.columns+"_fundtotal"

In [24]:
fund_total.reset_index().to_csv("fund_grants_total.csv", index=False)

In [44]:
fund2author = df[df.funder!='Not-Funded'][['author_distinct','funder','year']]
fund2author = fund2author.explode('author_distinct')
fund2author = fund2author.explode('funder')
fund2author = fund2author.groupby(['author_distinct','funder','year']).size().reset_index(name='fundcnt')

In [45]:
topcntrys=['United States','China','EU','United Kingdom','France','Germany']
fund2author['region'] = fund2author['funder'].apply(lambda x: x if x in topcntrys else "others")
fund2author.loc[fund2author['author_distinct']==fund2author['funder'],'region']='domestic' #assign "domestic"

In [46]:
fund2author = fund2author.groupby(['author_distinct','region','year'])['fundcnt'].sum().reset_index()

In [47]:
fund2author = fund2author.pivot_table(index=['author_distinct','year'],
                                      columns='region',values='fundcnt',fill_value=0)

fund2author=np.log(fund2author+1)
fund2author.columns = fund2author.columns+"_prev"
fund2author = fund2author.reset_index()

In [49]:
fund2author.to_csv("fund_previous.csv",index=False)